### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [ ]:
######
# will check parameter
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# word sample
word_sample = True  # True, False
word_sample_num = 20

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word
######

In [3]:
folder_name = "Turkish"
word_num = 28
adjust_sent_word_ratio = 66
adjust_videoid_text_word_ratio = 50
sample_sent_num = 3

#### Calculate Sentence Ratio By Word

In [4]:
#### common word for all language will test
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [5]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
39995,39995,00:01:15.200,00:01:18.840,çilek adam şip şap şop,iXAdoyurn18
39996,39996,00:01:19.000,00:01:23.240,burnu uzun lü lü lü,iXAdoyurn18
39997,39997,00:01:23.400,00:01:27.040,saçları rüzgar vu hu hu,iXAdoyurn18
39998,39998,00:01:27.620,00:01:30.720,kaşları keman gıy gıy gıy,iXAdoyurn18


In [6]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [7]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"bunlar, kamyon, kullanılıyor, özgür, büyük, ar...","için, daha"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"kolay, olmayacak, arka, tarafı, bağlamak, burcu",
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, arabaya, özgür, yarım, tur",
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"oldu, tamam, devam, et, burcu, biraz",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"kalınlığı, tekere, takacağız, artarsa, iki, şi...",ama
...,...,...,...,...,...,...,...,...
39995,39995,00:01:15.200,00:01:18.840,çilek adam şip şap şop,iXAdoyurn18,0.1,"çilek, şop, şap, adam, şip",
39996,39996,00:01:19.000,00:01:23.240,burnu uzun lü lü lü,iXAdoyurn18,0.1,"lü, uzun, burnu",
39997,39997,00:01:23.400,00:01:27.040,saçları rüzgar vu hu hu,iXAdoyurn18,0.1,"hu, vu, saçları, rüzgar",
39998,39998,00:01:27.620,00:01:30.720,kaşları keman gıy gıy gıy,iXAdoyurn18,0.1,"gıy, kaşları, keman",


In [8]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"bunlar, kamyon, kullanılıyor, özgür, büyük, ar...","için, daha"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"kolay, olmayacak, arka, tarafı, bağlamak, burcu",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, arabaya, özgür, yarım, tur",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"oldu, tamam, devam, et, burcu, biraz",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"kalınlığı, tekere, takacağız, artarsa, iki, şi...",ama
...,...,...,...,...,...,...,...,...
39995,39995,00:01:15.200,00:01:18.840,çilek adam şip şap şop,iXAdoyurn18,0.1,"çilek, şop, şap, adam, şip",NaN
39996,39996,00:01:19.000,00:01:23.240,burnu uzun lü lü lü,iXAdoyurn18,0.1,"lü, uzun, burnu",NaN
39997,39997,00:01:23.400,00:01:27.040,saçları rüzgar vu hu hu,iXAdoyurn18,0.1,"hu, vu, saçları, rüzgar",NaN
39998,39998,00:01:27.620,00:01:30.720,kaşları keman gıy gıy gıy,iXAdoyurn18,0.1,"gıy, kaşları, keman",NaN


In [9]:
df_sentence_ratio_result.to_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Sentence Ratio Result

In [10]:
#df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"kullanılıyor, özgür, araçlar, bunlar, büyük, n...","için, daha"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, kolay, tarafı, olmayacak, bağlamak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, özgür, tur, arabaya, yarım",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"burcu, et, oldu, biraz, tamam, devam",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, şimdilik, iki, tekere, takacağız, kalın...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"ellerde, umarız, yükselmez, güçlü, herkes, tek...","için, ve, bu"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"yararlanacağı, herkesin, gelişir, özgün, halde...","ve, daha, bir"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"zlediğiniz, teşekkürler, i",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, yararlandığım, kaynakları, olarak, ...",NaN


In [11]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,51,00:12:06.015,00:12:06.783,köpek değil bu,8V9tq1pe8eI,66.8,köpek,"bu, değil"
1,83,00:13:53.164,00:13:54.932,daha çok aşağıda,8V9tq1pe8eI,66.8,aşağıda,"daha, çok"
2,104,00:15:22.963,00:15:24.927,çok soruyorlar bu kadar,8V9tq1pe8eI,75.1,soruyorlar,"bu, çok, kadar"
3,151,00:19:13.279,00:19:14.815,gerçi bu da,8V9tq1pe8eI,66.8,gerçi,"bu, da"
4,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
...,...,...,...,...,...,...,...,...
66104,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
66105,3036751,00:23:49.370,00:23:55.040,bir şey gibi alın yok,sSTIfQUm18Q,80.1,alın,"şey, bir, gibi, yok"
66106,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
66107,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [12]:
df_adjust_sentence_ratio.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv", index=False)

In [13]:
df_adjust_sentence_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Videoid Text Ratio By Word

In [9]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,o,5013838
7,ben,4908913
8,de,4880315
9,çok,4852169


In [10]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...,...
39995,39995,00:01:15.200,00:01:18.840,çilek adam şip şap şop,iXAdoyurn18
39996,39996,00:01:19.000,00:01:23.240,burnu uzun lü lü lü,iXAdoyurn18
39997,39997,00:01:23.400,00:01:27.040,saçları rüzgar vu hu hu,iXAdoyurn18
39998,39998,00:01:27.620,00:01:30.720,kaşları keman gıy gıy gıy,iXAdoyurn18


In [17]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

,index,video_id,sentence
0,0,0ZPymcyoVSs,greyfurt g harfi merhaba çocuklar çocuklar bu ...
1,1,0_lJZgyPDTg,frambuaz f harfi merhaba çocuklar bugün sizi f...
2,2,0eFtntWEBAk,az sonra az önce mikrofonum kapanmış hepiniz h...
3,3,1R_1n6AMRpc,evet arkadaşlar hepinize selamlar biz sizlerle...
4,4,1sCtqOmOpMU,merhabalar gülün mutfak aşkı youtube kanalına ...
...,...,...,...
318,318,zJCqecTO148,evet hoş geldiniz hadi buraya gel hoş geldin b...
319,319,zQSwIKhtHrw,bende bende tamam bende ben ninjaya geliyorum ...
320,320,zd1S7DpPBwU,l harfi merhaba çocuklar bugünkü kahramanımız ...
321,321,zhj_xDOWxL0,herkese merhabalar gülün mutfak aşkı youtube k...


In [21]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [24]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence",f"{word_num}_word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,0ZPymcyoVSs,greyfurt g harfi merhaba çocuklar çocuklar bu ...,10.9,"geçer, üstüne, kısmına, i, içindeki, e, yapalı...","de, seni, ve, evet, için, mı, da, ben, sen, ka..."
1,0_lJZgyPDTg,frambuaz f harfi merhaba çocuklar bugün sizi f...,8.6,"yapar, ile, i, bak, seninle, abone, şşşttt, e,...","de, ve, evet, için, da, ama, sen, çok, var"
2,0eFtntWEBAk,az sonra az önce mikrofonum kapanmış hepiniz h...,4.5,"ki, olun, videoları, yapmaya, hileler, istiyor...","de, şey, değil, mı, da, onu, daha, ama, yok, b..."
3,1R_1n6AMRpc,evet arkadaşlar hepinize selamlar biz sizlerle...,1.9,"üstüne, köyü, bunların, modu, halı, hava, fazl...","onu, var, ve, evet, değil, için, o, da, yok, b..."
4,1sCtqOmOpMU,merhabalar gülün mutfak aşkı youtube kanalına ...,4.2,"buram, un, yapmaya, toz, portakal, çıktı, tane...","de, ve, bu, için, o, da, daha, yok, ben, kadar..."
...,...,...,...,...,...
318,zJCqecTO148,evet hoş geldiniz hadi buraya gel hoş geldin b...,4.5,"burcuyum, herhangi, olun, bakımlı, gidiyorsun,...","bana, var, ve, evet, değil, için, da, yok, bir..."
319,zQSwIKhtHrw,bende bende tamam bende ben ninjaya geliyorum ...,3.3,"hakikaten, sercan, ki, azıcık, çakar, eldiven,...","bana, onu, var, ve, evet, değil, için, o, da, ..."
320,zd1S7DpPBwU,l harfi merhaba çocuklar bugünkü kahramanımız ...,6.1,"acele, yaptığı, vitaminler, lahanadaki, hastal...","ve, için, ben, bir, var"
321,zhj_xDOWxL0,herkese merhabalar gülün mutfak aşkı youtube k...,2.5,"sote, ki, aldıktan, kullanmak, hazırlamış, sar...","de, ve, bu, için, o, da, daha, ama, ben, kadar..."


In [30]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,0ZPymcyoVSs,greyfurt g harfi merhaba çocuklar çocuklar bu ...,10.9,"geçer, üstüne, kısmına, i, içindeki, e, yapalı...","de, seni, ve, evet, için, mı, da, ben, sen, ka..."
1,0_lJZgyPDTg,frambuaz f harfi merhaba çocuklar bugün sizi f...,8.6,"yapar, ile, i, bak, seninle, abone, şşşttt, e,...","de, ve, evet, için, da, ama, sen, çok, var"
2,0eFtntWEBAk,az sonra az önce mikrofonum kapanmış hepiniz h...,4.5,"ki, olun, videoları, yapmaya, hileler, istiyor...","de, şey, değil, mı, da, onu, daha, ama, yok, b..."
3,1R_1n6AMRpc,evet arkadaşlar hepinize selamlar biz sizlerle...,1.9,"üstüne, köyü, bunların, modu, halı, hava, fazl...","onu, var, ve, evet, değil, için, o, da, yok, b..."
4,1sCtqOmOpMU,merhabalar gülün mutfak aşkı youtube kanalına ...,4.2,"buram, un, yapmaya, toz, portakal, çıktı, tane...","de, ve, bu, için, o, da, daha, yok, ben, kadar..."
...,...,...,...,...,...
318,zJCqecTO148,evet hoş geldiniz hadi buraya gel hoş geldin b...,4.5,"burcuyum, herhangi, olun, bakımlı, gidiyorsun,...","bana, var, ve, evet, değil, için, da, yok, bir..."
319,zQSwIKhtHrw,bende bende tamam bende ben ninjaya geliyorum ...,3.3,"hakikaten, sercan, ki, azıcık, çakar, eldiven,...","bana, onu, var, ve, evet, değil, için, o, da, ..."
320,zd1S7DpPBwU,l harfi merhaba çocuklar bugünkü kahramanımız ...,6.1,"acele, yaptığı, vitaminler, lahanadaki, hastal...","ve, için, ben, bir, var"
321,zhj_xDOWxL0,herkese merhabalar gülün mutfak aşkı youtube k...,2.5,"sote, ki, aldıktan, kullanmak, hazırlamış, sar...","de, ve, bu, için, o, da, daha, ama, ben, kadar..."


In [ ]:
df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv", index=False)

In [25]:
# 2 Way
#df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
#df_videoid_sentence

In [26]:
#def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
#    '''
#    videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#    '''
#    word_set = set(df_word[f"{word_column_name}"].to_list())
#    for i in range(len(df_videoid_sentence)):
#        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
#        text_word = re.findall(r"\w+", text, re.UNICODE)
#        text_word_set = set(text_word)
#        intersect_word = list(word_set.intersection(text_word_set))
#        different_word = list(text_word_set.difference(word_set))
#        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = round(((len(intersect_word)/len(text_word)+0.001)*100),)
#        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
#        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
#        
#    return df_videoid_sentence

In [27]:
#df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", word_num)
#df_text_ratio_result

In [28]:
#df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
#df_text_ratio_result 

In [29]:
#df_text_ratio_result.to_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv", index=False)

#### Select Adjust Videoid Text Ratio Result

In [4]:
#df_text_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv")
df_text_ratio_result = pd.read_csv(f"Youtube_Videoid_Text_{word_num}_Word_All_Ratio_Result.csv")
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,007uL_OBtPE,merhaba arkadaşlar bugün sizlere karadeniz yör...,7.0,"sürekli, başlıyorum, olan, çayın, şunu, tavsiy...","için, bu, var, gibi, kadar, ve, bunu, de, ama,..."
1,00Dl6tNNlZ8,kontrol tamam galiba değil mi eee değerli bası...,2.0,"uçaklarla, amacımız, yapalım, elimizden, hesab...","değil, onu, ama, çok, bu, ne, gibi, mi, ben, b..."
2,00QJHZSg0UE,ioğluyla biraz vakit geçirsini ievlat bui idün...,11.0,"hem, değişilmezi, iyoksa, hasretimsini, gereği...","ne, sen, mi, ben, de"
3,00b3ov_NFSI,evleniyorsun kocam diyorsun aynı yastığa baş k...,14.0,"denen, baş, boşanmış, i, koyuyorsun, mesaileri...","bu, ne, beni, değil, sen, gibi, var, kadar, on..."
4,00fr8yyBQYo,bill gates and george soros developed a new co...,0.0,"games, return, am, guard, coincidence, needs, ...",NaN
...,...,...,...,...,...
17990,zzOL9mRRU0c,hi kaye di yari merhaba arkadaşlar hikaye diya...,3.0,"pazardan, atışlarının, elma, kalp, güçlü, yayı...","için, iyi, bu, ne, beni, gibi, var, de, bana, ..."
17991,zzWQXUKr9OA,bak baştan anlaşalım ya bu cinayete de diğer c...,15.0,"bilmem, en, baştan, i, kızınız, dışında, lan, ...","bu, gibi, bana, mi, ben, seni, de, da"
17992,zzXTIIo7EvM,ev avcısı kuşadası değirmendere de bugün örnek...,1.0,"aşağıda, yaşıyorlar, dokunuşlar, yerden, ulaşı...","onu, ama, çok, bu, ne, gibi, bana, mi, ben, bi..."
17993,zzZfnY7Zjf0,ben arjantinli bir yapımcıyım ve taşınabilirli...,14.0,"işler, yapımcısı, setteyken, senaryoyu, edebil...","için, kadar, ve, ben, bir, çok"


In [22]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result[f"{word_num}_word_ratio"] >= adjust_videoid_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,2LqpcYH8BVk,bu,100.0,NaN,bu
1,3ceSnmV4vPc,ben burada,50.0,burada,ben
2,CqQ7n02vocc,devamı var,50.0,devamı,var
3,GiJuh_GPHl0,bu köyün,50.0,köyün,bu
4,Jr3tr19glro,iyi seyirler,50.0,seyirler,iyi
5,Jv5n18MD32M,çok basit,50.0,basit,çok
6,S4F9FP1e2ps,ama sen çok pahalı bir kedi fiyatı,57.0,"pahalı, fiyatı, kedi","çok, sen, bir, ama"
7,WJY9mOwEFSE,karşınızda adam mı var hemen uygun bir pozisyo...,50.0,"karşınızda, uygun, pozisyonda, hemen, adam, vurun","mı, var, onu, kadar, bu, bir"
8,bw0DBP0oMkA,reklamlarınız için,50.0,reklamlarınız,için
9,pqKKGDNxyQ4,iyi seyirler,50.0,seyirler,iyi


In [23]:
df_adjust_text_ratio.to_csv(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.csv", index=False)

In [24]:
df_adjust_text_ratio.to_excel(f"Youtube_Videoid_{word_num}_Word_{adjust_videoid_text_word_ratio}_Ratio_Text.xlsx", index=False)

In [6]:
df_text_ratio_result.sort_values(by="28_word_ratio", ascending=False)

,video_id,sentence,28_word_ratio,different_word,intersect_word
655,2LqpcYH8BVk,bu,100.0,NaN,bu
8043,S4F9FP1e2ps,ama sen çok pahalı bir kedi fiyatı,57.0,"pahalı, kedi, fiyatı","ama, çok, sen, bir"
3670,CqQ7n02vocc,devamı var,50.0,devamı,var
4791,GiJuh_GPHl0,bu köyün,50.0,köyün,bu
11155,bw0DBP0oMkA,reklamlarınız için,50.0,reklamlarınız,için
...,...,...,...,...,...
3326,BgnIou8fnVo,deni zli dans okulu amp organi zasyon,0.0,"dans, zli, organi, amp, okulu, zasyon, deni",NaN
15065,ph2dOWBiOng,mansur yavaşankara büyükşehir belediyesiankara...,0.0,"meclis, meclismansur, büyükşehir, tartışmamans...",NaN
15066,ph955Tx8v34,menekşe bakimi i le i lgi li bi li nmeyenler m...,0.0,"i, le, li, bi, menekşe, yaz, bakimi, lgi, nmey...",NaN
6679,NNk40xY5sV0,i zlemekte olduğunuz nar bahçesinde rekor daml...,0.0,"olduğunuz, i, bilgi, zlemekte, kullanılmıştır,...",NaN


#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [8]:
#df_sentence_ratio_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_All_Ratio_Result.csv")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"kullanılıyor, özgür, araçlar, bunlar, büyük, n...","için, daha"
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, kolay, tarafı, olmayacak, bağlamak",NaN
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, özgür, tur, arabaya, yarım",NaN
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"burcu, et, oldu, biraz, tamam, devam",daha
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, şimdilik, iki, tekere, takacağız, kalın...",ama
...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"ellerde, umarız, yükselmez, güçlü, herkes, tek...","için, ve, bu"
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"yararlanacağı, herkesin, gelişir, özgün, halde...","ve, daha, bir"
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"zlediğiniz, teşekkürler, i",için
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, yararlandığım, kaynakları, olarak, ...",NaN


In [9]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].astype("object")
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].astype("object")

In [14]:
#df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,51,00:12:06.015,00:12:06.783,köpek değil bu,8V9tq1pe8eI,66.8,köpek,"bu, değil"
1,83,00:13:53.164,00:13:54.932,daha çok aşağıda,8V9tq1pe8eI,66.8,aşağıda,"daha, çok"
2,104,00:15:22.963,00:15:24.927,çok soruyorlar bu kadar,8V9tq1pe8eI,75.1,soruyorlar,"bu, çok, kadar"
3,151,00:19:13.279,00:19:14.815,gerçi bu da,8V9tq1pe8eI,66.8,gerçi,"bu, da"
4,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
...,...,...,...,...,...,...,...,...
66104,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
66105,3036751,00:23:49.370,00:23:55.040,bir şey gibi alın yok,sSTIfQUm18Q,80.1,alın,"şey, bir, gibi, yok"
66106,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
66107,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [15]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [16]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,51,00:12:06.015,00:12:06.783,köpek değil bu,8V9tq1pe8eI,66.8,köpek,"bu, değil",1
1,83,00:13:53.164,00:13:54.932,daha çok aşağıda,8V9tq1pe8eI,66.8,aşağıda,"daha, çok",1
2,104,00:15:22.963,00:15:24.927,çok soruyorlar bu kadar,8V9tq1pe8eI,75.1,soruyorlar,"bu, çok, kadar",1
3,151,00:19:13.279,00:19:14.815,gerçi bu da,8V9tq1pe8eI,66.8,gerçi,"bu, da",1
4,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet,1
...,...,...,...,...,...,...,...,...,...
66104,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben,1
66105,3036751,00:23:49.370,00:23:55.040,bir şey gibi alın yok,sSTIfQUm18Q,80.1,alın,"şey, bir, gibi, yok",1
66106,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu,1
66107,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen,1


In [17]:
df_adjust_ratio_flag = pd.merge(df_sentence_ratio_result , df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI,22.3,"kullanılıyor, özgür, araçlar, bunlar, büyük, n...","için, daha",0.0
1,1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI,0.1,"burcu, arka, kolay, tarafı, olmayacak, bağlamak",NaN,0.0
2,2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI,0.1,"attıracağım, özgür, tur, arabaya, yarım",NaN,0.0
3,3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI,11.2,"burcu, et, oldu, biraz, tamam, devam",daha,0.0
4,4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI,11.2,"burcu, şimdilik, iki, tekere, takacağız, kalın...",ama,0.0
...,...,...,...,...,...,...,...,...,...
3036932,3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q,25.1,"ellerde, umarız, yükselmez, güçlü, herkes, tek...","için, ve, bu",0.0
3036933,3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q,25.1,"yararlanacağı, herkesin, gelişir, özgün, halde...","ve, daha, bir",0.0
3036934,3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q,25.1,"zlediğiniz, teşekkürler, i",için,0.0
3036935,3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q,0.1,"belirttim, yararlandığım, kaynakları, olarak, ...",NaN,0.0


In [18]:
df_adjust_ratio_flag.with_word.value_counts()

0.0    2970828
1.0      66109
Name: with_word, dtype: int64

In [19]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [20]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

,index,length
0,1972884,7
1,1447811,6
2,3297,6
3,1371269,6
4,2042051,6
...,...,...
62297,1065960,1
62298,1065994,1
62299,1066126,1
62300,1066129,1


In [21]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,1972884,00:20:52.072,00:20:52.816,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
1,1972885,00:20:53.924,00:20:54.724,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
2,1972886,00:20:58.514,00:20:59.064,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
3,1972887,00:21:04.290,00:21:04.840,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
4,1972888,00:21:15.614,00:21:16.164,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
...,...,...,...,...,...,...,...,...,...
66104,1065960,00:04:54.371,00:04:55.756,miran iyi mi,3EBHjo8rngQ,66.8,miran,"iyi, mi",1.0
66105,1065994,00:06:38.514,00:06:40.028,ne yaptın sen,3EBHjo8rngQ,66.8,yaptın,"sen, ne",1.0
66106,1066126,00:15:54.730,00:15:56.072,bunu bana mı söylüyorsun,3EBHjo8rngQ,75.1,söylüyorsun,"mı, bunu, bana",1.0
66107,1066129,00:16:18.295,00:16:19.755,sendin değil mi,3EBHjo8rngQ,66.8,sendin,"mi, değil",1.0


In [ ]:
#df_one_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentences_And_Others.csv", index=False)

In [24]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

,index,length
0,1972884,7
1,1447811,6
2,3297,6
3,1371269,6
4,2042051,6
...,...,...
3384,1883381,2
3385,1883800,2
3386,813688,2
3387,2058797,2


In [25]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,1972884,00:20:52.072,00:20:52.816,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
1,1972885,00:20:53.924,00:20:54.724,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
2,1972886,00:20:58.514,00:20:59.064,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
3,1972887,00:21:04.290,00:21:04.840,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
4,1972888,00:21:15.614,00:21:16.164,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
...,...,...,...,...,...,...,...,...,...
7191,813689,00:56:44.758,00:56:45.666,evet,LFHDU8S0yFg,100.1,NaN,evet,1.0
7192,2058797,00:03:23.995,00:03:24.804,ne,8G6vBOOtiy0,100.1,NaN,ne,1.0
7193,2058798,00:03:24.814,00:03:25.907,ne söylüyorsun bana,8G6vBOOtiy0,66.8,söylüyorsun,"bana, ne",1.0
7194,1814229,01:15:38.681,01:15:39.812,bir sıkıntı mı var,jluoByJbQgg,75.1,sıkıntı,"var, mı, bir",1.0


In [39]:
df_one_sequential_all.to_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv", index=False)

In [26]:
df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [41]:
#df_one_sequential_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Only_Sequential_Sentences.csv")
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,1972884,00:20:52.072,00:20:52.816,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
1,1972885,00:20:53.924,00:20:54.724,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
2,1972886,00:20:58.514,00:20:59.064,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
3,1972887,00:21:04.290,00:21:04.840,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
4,1972888,00:21:15.614,00:21:16.164,hayır,LVPw3VkNyME,100.1,NaN,hayır,1.0
...,...,...,...,...,...,...,...,...,...
2215,13030,00:18:10.840,00:18:11.560,hayır,CNPzcVvZ1nU,100.1,NaN,hayır,1.0
2216,2997400,00:52:23.049,00:52:23.848,evet,LmY6NZwmSa4,100.1,NaN,evet,1.0
2217,2997401,00:52:23.848,00:52:24.348,evet,LmY6NZwmSa4,100.1,NaN,evet,1.0
2218,1232055,00:47:26.871,00:47:28.171,evet,AxEi0Uy8gMQ,100.1,NaN,evet,1.0


In [42]:
def text_to_list_df(df_target, target_column): 
    '''
    df_target include dict value columns, target_column occurs dict \n
    text_to_list_df(df_one_sequential_all, "intersect_word")
    '''
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [43]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

,intersect_word_alone,count
0,evet,445
1,ne,294
2,ben,268
3,hayır,239
4,yok,198
5,bu,187
6,sen,166
7,ama,132
8,var,109
9,mi,109


In [ ]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [44]:
#df_adjust_ratio = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/5-Youtube Sentence Adjust Words Ratio/Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio = pd.read_csv(f"Youtube_Sentence_{word_num}_Word_{adjust_sent_word_ratio}_Ratio_Sentences.csv")
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,222,00:27:46.048,00:27:46.560,evet,8V9tq1pe8eI,100.1,NaN,evet
1,605,00:10:35.360,00:10:36.520,evet,cQjX9DzXTw0,100.1,NaN,evet
2,1906,00:01:25.760,00:01:26.528,gibi,owluCDXgJSE,100.1,NaN,gibi
3,1923,00:02:02.368,00:02:02.880,ve,owluCDXgJSE,100.1,NaN,ve
4,2018,00:05:33.823,00:05:37.151,ve,owluCDXgJSE,100.1,NaN,ve
...,...,...,...,...,...,...,...,...
32045,3036709,00:17:25.970,00:17:26.000,bu,sSTIfQUm18Q,100.1,NaN,bu
32046,3036732,00:20:21.720,00:20:21.780,ben,sSTIfQUm18Q,100.1,NaN,ben
32047,3036760,00:25:30.370,00:25:30.400,bu,sSTIfQUm18Q,100.1,NaN,bu
32048,3036762,00:25:54.640,00:25:54.700,sen,sSTIfQUm18Q,100.1,NaN,sen


In [45]:
df_adjust_ratio["different_word"] = df_adjust_ratio["different_word"].astype("object")
df_adjust_ratio["intersect_word"] = df_adjust_ratio["intersect_word"].astype("object")

In [46]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")[f"{word_num}_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={f"{word_num}_word_ratio":f"{word_num}_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by=f"{word_num}_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

,video_id,28_word_ratio_count
0,eHtgG748qhI,140
1,qQUZ5d5mpWk,118
2,vLkbW6uukDU,110
3,WcRHKUhrmHQ,104
4,PmEa9xL7awE,84
...,...,...
4899,TAVSThqKUfw,1
4900,TBCr2GWYiBU,1
4901,TMLCOvRyZE4,1
4902,TQhN5dJS35A,1


In [ ]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{adjust_word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Find Word In Sentence Time Interval

In [47]:
df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_adjust_word = df_word.iloc[0:word_num,]
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [48]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,00:07:51.970,00:07:52.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,00:07:52.470,00:08:02.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,00:08:02.498,00:08:04.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,00:08:04.178,00:08:08.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [49]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [50]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3036932,471.970,472.470,umarız ki bu büyük ve güçlü teknoloji yanlış e...,YFFJ5FyZj4Q
3036933,472.470,482.304,daha faydalı ve özgün kurumlarda herkesin eşit...,YFFJ5FyZj4Q
3036934,482.498,484.178,i zlediğiniz için teşekkürler,YFFJ5FyZj4Q
3036935,484.178,488.089,yararlandığım kaynakları açıklamada link olara...,YFFJ5FyZj4Q


In [51]:
word_list = df_adjust_word.iloc[:,0].values.tolist()

In [52]:
df_word_result = pd.DataFrame()
for i in word_list:
    try:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].sample(sample_sent_num)
    except:
        word_in_video = df_youtube_sentence[df_youtube_sentence.sentence.str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(sample_sent_num) 
    word_in_video.insert(0,"word",i)
    df_word_result = pd.concat([df_word_result,word_in_video], axis=0)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result = df_word_result.sort_values(by=["video_id","start_time"], ascending=True)
df_word_result.reset_index(drop=True, inplace=True)
df_word_result

,word,start_time,end_time,sentence,video_id
0,beni,1875.712,1879.040,emir toplantı yüzünden beni asla affetmeyecek,1EEZhYHOqvU
1,kadar,451.120,453.560,çalışabildiğimiz kadar çalışmamız lazım,1Y5ZM9u13x0
2,onu,1139.760,1141.040,onu hiç görmedim,7Pt6Kj7ZGxc
3,bu,112.869,115.803,ki bu zaten bu zaten buradaki arkadaşlarım hiç...,7g8_EncEuh4
4,bunu,183.000,186.200,kimse bunu yapamadı yani ben görmedim yapan va...,BPWJGcTkw28
5,ben,1231.436,1233.023,tamam ben ikna oldum bravo evet,GQgS2ruEJAs
6,ve,177.192,179.618,ve kenar kısımlarından,IcwqolOEnnE
7,için,847.272,850.725,mehmet ali birandın sunduğu ana haber bülteni ...,J8GHeYett3k
8,var,5238.400,5239.580,sen var ya kot kafalısın,KHyLRPII8ow
9,evet,4077.280,4078.320,evet,KVCZbb3svSc


In [53]:
word_time_loc_list = []
def word_time_loc(df):
    for i in range(len(df)):
        word = df.loc[i,"word"]
        start_time = df.loc[i,"start_time"]
        end_time = df.loc[i,"end_time"]
        sentence = df.loc[i,"sentence"]
        video_id = df.loc[i,"video_id"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=["word","start_time","end_time","sentence","video_id"])

    return df_word_time_loc

In [54]:
df_word_time_loc_result = word_time_loc(df_word_result)
df_word_time_loc_result

,word,start_time,end_time,sentence,video_id
0,beni,1877.339022,1877.782756,emir toplantı yüzünden beni asla affetmeyecek,1EEZhYHOqvU
1,kadar,452.121026,452.558974,çalışabildiğimiz kadar çalışmamız lazım,1Y5ZM9u13x0
2,onu,1139.760000,1140.080000,onu hiç görmedim,7Pt6Kj7ZGxc
3,bu,112.949384,113.110151,ki bu zaten bu zaten buradaki arkadaşlarım hiç...,7g8_EncEuh4
4,bunu,183.246154,183.541538,kimse bunu yapamadı yani ben görmedim yapan va...,BPWJGcTkw28
5,ben,1231.691968,1231.947935,tamam ben ikna oldum bravo evet,GQgS2ruEJAs
6,ve,177.192000,177.522818,ve kenar kısımlarından,IcwqolOEnnE
7,için,850.379700,850.725000,mehmet ali birandın sunduğu ana haber bülteni ...,J8GHeYett3k
8,var,5238.547500,5238.793333,sen var ya kot kafalısın,KHyLRPII8ow
9,evet,4077.280000,4078.320000,evet,KVCZbb3svSc


In [55]:
time_shift = 0.3
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_time_loc_result

,word,start_time,end_time,sentence,video_id
0,beni,1877.039022,1878.082756,emir toplantı yüzünden beni asla affetmeyecek,1EEZhYHOqvU
1,kadar,451.821026,452.858974,çalışabildiğimiz kadar çalışmamız lazım,1Y5ZM9u13x0
2,onu,1139.460000,1140.380000,onu hiç görmedim,7Pt6Kj7ZGxc
3,bu,112.649384,113.410151,ki bu zaten bu zaten buradaki arkadaşlarım hiç...,7g8_EncEuh4
4,bunu,182.946154,183.841538,kimse bunu yapamadı yani ben görmedim yapan va...,BPWJGcTkw28
5,ben,1231.391968,1232.247935,tamam ben ikna oldum bravo evet,GQgS2ruEJAs
6,ve,176.892000,177.822818,ve kenar kısımlarından,IcwqolOEnnE
7,için,850.079700,851.025000,mehmet ali birandın sunduğu ana haber bülteni ...,J8GHeYett3k
8,var,5238.247500,5239.093333,sen var ya kot kafalısın,KHyLRPII8ow
9,evet,4076.980000,4078.620000,evet,KVCZbb3svSc


In [56]:
df_word_time_loc_result.start_time = df_word_time_loc_result.start_time.apply(lambda x: round(x))
df_word_time_loc_result.end_time = df_word_time_loc_result.end_time.apply(lambda x: round(x))
df_word_time_loc_result 

,word,start_time,end_time,sentence,video_id
0,beni,1877,1878,emir toplantı yüzünden beni asla affetmeyecek,1EEZhYHOqvU
1,kadar,452,453,çalışabildiğimiz kadar çalışmamız lazım,1Y5ZM9u13x0
2,onu,1139,1140,onu hiç görmedim,7Pt6Kj7ZGxc
3,bu,113,113,ki bu zaten bu zaten buradaki arkadaşlarım hiç...,7g8_EncEuh4
4,bunu,183,184,kimse bunu yapamadı yani ben görmedim yapan va...,BPWJGcTkw28
5,ben,1231,1232,tamam ben ikna oldum bravo evet,GQgS2ruEJAs
6,ve,177,178,ve kenar kısımlarından,IcwqolOEnnE
7,için,850,851,mehmet ali birandın sunduğu ana haber bülteni ...,J8GHeYett3k
8,var,5238,5239,sen var ya kot kafalısın,KHyLRPII8ow
9,evet,4077,4079,evet,KVCZbb3svSc


In [ ]:
#df_word_time_loc_result.to_excel(f"Youtube_Sentence_{word_num}_Word_Time_Interval.xlsx", index=False)